<h1>Model Training</h1>

In this notebook, we will use the Amazon SageMaker built-in XGBoost algorithm (https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) to train a simple binary classification model, using the pre-processed data generated in the previous step by the AWS Glue job. Let's define some variables first.

<span style="color: red"> Please replace your initials in the bucket_name variable defined in next cell.</span>

In [ ]:
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print(region)
print(role)

# replace [your-initials] according to the bucket name you have defined.
bucket_name = 'endtoendml-workshop-[your-initials]'
key_prefix = 'data/preprocessed'

Now we take a look at our preprocessed data, which have already been split into training and validation sets by the AWS Glue job. In order to do that, we first download a preprocessed training file from Amazon S3 to the local notebook file system.

In [ ]:
import boto3

# The name of the file has been set by AWS Glue job in the previous notebook.
train_file_name = 'part-00000'
train_file_key = '{0}/train/{1}'.format(key_prefix, train_file_name)

s3 = boto3.resource('s3')
s3.Bucket(bucket_name).download_file(train_file_key, train_file_name)

After the file has been download, we can use Pandas to read the CSV and display the first 10 rows. You can immediately notice that categorical features have been one-hot encoded according to the feature engineering actions executed in the previous step.

In [ ]:
import pandas
import numpy

df = pandas.read_csv(train_file_name, header=None)
df.head(10)

In order to start training, we need to specify the location of the docker container that will be used for training.
Docker Registry paths for Amazon algorithms are specified here: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

By the way, we can use a utility function of the Amazon SageMaker Python SDK to get the path.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version="latest")
print(container)

We can now start training, by specifying the input and output settings and the required hyperparameters. You can find the list of the supported hyperparameters for the XGBoost algorithm here: https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html.

You can also try running the following cell multiple times changing hyperparameters or other settings like the number of instances to be used for training, since XGBoost can be parallelized.

In [ ]:
import sagemaker

output_location = 's3://{0}/output'.format(bucket_name)

est = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m5.2xlarge',
                                    output_path=output_location,
                                    base_job_name='predmain-train-xgb')

est.set_hyperparameters(objective='reg:logistic',
                        max_depth=4,
                        num_round=20)

train_config = sagemaker.session.s3_input('s3://{0}/{1}/train/'.format(
    bucket_name, key_prefix), content_type='text/csv')
val_config = sagemaker.session.s3_input('s3://{0}/{1}/val/'.format(
    bucket_name, key_prefix), content_type='text/csv')

est.fit({'train': train_config, 'validation': val_config })

After the training is completed, the serialized model will be saved in the S3 output_location defined above.
You can now move to the next notebook in the **04_deploy_model** folder to see how to use that model for inference.